In [ ]:
# ============================================================
# MULTIMODAL VOICE AUTHENTICATION SYSTEM (Console Feedback)
# Author: Ivonne Chia | NTU DIP Project E012
# Version: v2.1 – Enhanced Robustness, UX & Logging
# ============================================================

import os, json, random, time, shutil, csv
from datetime import datetime
import numpy as np, torch, torchaudio, sounddevice as sd, speech_recognition as sr, wavio
from sklearn.metrics.pairwise import cosine_similarity
from speechbrain.inference import SpeakerRecognition
import tkinter as tk
from tkinter import filedialog, ttk

# ============================================================
# SETUP
# ============================================================

for f in ["data/registrations", "data/auth", "pretrained_models", "logs"]:
    os.makedirs(f, exist_ok=True)

print("✅ Environment initialized.")
print("🔄 Loading ECAPA-TDNN model…")
verifier = SpeakerRecognition.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="pretrained_models/ecapa"
)
print("✅ ECAPA-TDNN loaded.\n")

TARGET_SR = 16000

# ============================================================
# HELPERS
# ============================================================

def ensure_mono_16k(sig: torch.Tensor, sr: int):
    """Ensure audio is mono and resampled to 16kHz."""
    if sig.ndim == 2 and sig.shape[0] > 1:
        sig = sig.mean(dim=0, keepdim=True)
    elif sig.ndim == 1:
        sig = sig.unsqueeze(0)
    if sr != TARGET_SR:
        sig = torchaudio.functional.resample(sig, sr, TARGET_SR)
    return sig

def extract_embedding(path: str) -> np.ndarray:
    """Extract normalized ECAPA embedding from WAV."""
    try:
        s, sr = torchaudio.load(path)
        s = ensure_mono_16k(s, sr)
        emb = verifier.encode_batch(s).squeeze().detach().cpu().numpy().flatten()
        emb = emb / np.linalg.norm(emb)  # ✅ L2-normalized
        return emb
    except Exception as e:
        print(f"⚠️ Embedding failed: {e}")
        return np.zeros((192,), np.float32)

def normalize_audio(audio, target_dbfs=-20.0):
    """Normalize loudness."""
    rms = np.sqrt(np.mean(audio**2) + 1e-9)
    target = 10 ** (target_dbfs / 20)
    audio = audio - np.mean(audio)  # remove DC offset
    return np.clip(audio * (target / rms), -1, 1).astype(np.float32)

def generate_prompt(n=5):
    """Generate random n-digit authentication string."""
    return ''.join(str(random.randint(0, 9)) for _ in range(n))

def transcribe_audio(path: str):
    """ASR with ambient noise adjustment."""
    r = sr.Recognizer()
    with sr.AudioFile(path) as src:
        r.adjust_for_ambient_noise(src, duration=0.5)
        src.seek(0)
        audio = r.record(src)
    try:
        return r.recognize_google(audio).lower()
    except sr.UnknownValueError:
        return ""
    except sr.RequestError as e:
        print(f"⚠️ ASR request failed: {e}")
        return ""

def text_to_digits(txt: str) -> str:
    """Convert spoken digits (words or numerals) into numeric string."""
    txt = txt.strip()
    if txt.isdigit():
        return txt
    if all(ch.isdigit() or ch.isspace() for ch in txt):
        return txt.replace(" ", "")

    mapping = {
        "zero":"0", "oh":"0", "o":"0",
        "one":"1", "two":"2", "to":"2", "too":"2",
        "three":"3", "four":"4", "for":"4",
        "five":"5", "six":"6",
        "seven":"7", "eight":"8", "nine":"9"
    }
    words = txt.lower().split()
    return ''.join(mapping.get(w, "") for w in words)

# ============================================================
# RECORDING GUI
# ============================================================

def record_audio(prompt: str, fs=TARGET_SR, duration=5):
    """Small GUI window to record audio."""
    root = tk.Tk()
    root.title("Voice Recording")
    root.geometry("430x250"); root.resizable(False, False)

    tk.Label(root, text="Say these digits:", font=("Arial", 12)).pack(pady=(15, 0))
    tk.Label(root, text=prompt, font=("Arial", 20, "bold")).pack(pady=(4, 10))
    status = tk.Label(root, text="Preparing…", font=("Arial", 11)); status.pack()
    bar = ttk.Progressbar(root, orient="horizontal", length=360, maximum=duration*10)
    bar.pack(pady=8)

    root.update(); time.sleep(0.5)
    for i in range(3, 0, -1):
        status.config(text=f"🎬 Recording starts in {i}…"); root.update(); time.sleep(1)

    status.config(text="🎙️ Recording…"); root.update()
    audio = sd.rec(int(duration*fs), samplerate=fs, channels=1, dtype="float32")
    for i in range(duration*10):
        bar["value"] = i+1; root.update_idletasks(); time.sleep(0.1)
    sd.wait()

    audio = normalize_audio(audio)
    out = f"data/auth/auth_{datetime.now().strftime('%Y%m%d_%H%M%S')}.wav"
    wavio.write(out, audio, fs, sampwidth=2)
    status.config(text="✅ Done!"); root.update(); time.sleep(0.5)
    root.destroy()
    return out

# ============================================================
# REGISTRATION
# ============================================================

def register_user():
    """Register a new user by uploading 3 WAV samples."""
    name = input("👤 Username → ").strip()
    if not name:
        print("❌ Cancelled."); return
    user_dir = f"data/registrations/{name}"; os.makedirs(user_dir, exist_ok=True)

    paths = []
    for i in range(3):
        print(f"📁 Upload WAV {i+1}")
        tk.Tk().withdraw()
        f = filedialog.askopenfilename(title=f"Select recording {i+1}", filetypes=[("WAV","*.wav")])
        if not f:
            print("❌ Cancelled."); return
        try:
            info = torchaudio.info(f)
            if info.num_frames == 0:
                print("⚠️ Invalid or empty audio file."); return
        except Exception:
            print("⚠️ Invalid file."); return

        dest = os.path.join(user_dir, f"sample_{i+1}.wav")
        shutil.copy(f, dest)
        print("✅", dest)
        paths.append(dest)

    print("🔍 Extracting embeddings…")
    embeddings = [extract_embedding(p) for p in paths]
    avg = np.mean(embeddings, axis=0)

    # dynamic threshold
    scores = [cosine_similarity([avg], [e])[0][0] for e in embeddings]
    user_threshold = max(0.7, np.mean(scores) - 0.05)

    db = "storage_users.json"
    data = {}
    if os.path.exists(db):
        try:
            with open(db) as f:
                data = json.load(f)
        except:
            data = {}

    data[name] = {
        "embedding": avg.tolist(),
        "threshold": user_threshold,
        "registered_at": datetime.now().isoformat(),
        "num_samples": len(paths)
    }

    with open(db, "w") as f:
        json.dump(data, f, indent=4)

    print(f"✅ Registered '{name}' ({len(paths)} samples, threshold={user_threshold:.2f}).\n")

# ============================================================
# AUTHENTICATION
# ============================================================

def authenticate_user(default_threshold=0.8, duration=10):
    """Authenticate an existing user by recording new audio."""
    name = input("👤 Username → ").strip()
    if not name:
        print("❌ Cancelled."); return

    db = "storage_users.json"
    if not os.path.exists(db):
        print("❌ No registered users."); return

    try:
        with open(db) as f:
            users = json.load(f)
    except:
        users = {}

    if name not in users:
        print(f"❌ '{name}' not found."); return

    prompt = generate_prompt()
    print(f"\n🗣️ Say the digits → {prompt}\n")

    audio_path = record_audio(prompt, duration=duration)

    print("🎧 Analyzing voice...")
    reg = np.array(users[name]["embedding"], dtype=np.float32).flatten()
    test = extract_embedding(audio_path)

    if not test.any():
        print("⚠️ Audio too short / silent. Try again."); return

    voice = float(cosine_similarity([test], [reg])[0][0])
    transcribed = transcribe_audio(audio_path)
    recognized = text_to_digits(transcribed)
    content = 1.0 if recognized == prompt else 0.0

    # Adaptive weighting
    voice_weight = 0.8 if voice > 0.85 else 0.6
    final = voice_weight * voice + (1 - voice_weight) * content

    threshold = users[name].get("threshold", default_threshold)

    print("──────────────────────────────")
    print(f"👤 {name}")
    print(f"🎧 Voice Score: {voice:.3f}")
    print(f"🗣️ ASR Text: {transcribed or '(none)'}")
    print(f"🔢 Recognized: {recognized or '(none)'}")
    print(f"🎯 Content Match: {'✅' if content else '❌'}")
    print(f"📊 Final Score: {final:.3f} (Threshold={threshold:.2f})")
    print("──────────────────────────────")

    result = "PASS" if final >= threshold else "FAIL"
    print("✅ ACCESS GRANTED — identity verified.\n" if result == "PASS"
          else "🚫 ACCESS DENIED — authentication failed.\n")

    # Logging
    with open("logs/auth_log.csv", "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            datetime.now().isoformat(), name, prompt,
            f"{voice:.3f}", recognized, f"{final:.3f}", result
        ])

# ============================================================
# MAIN
# ============================================================

if __name__ == "__main__":
    print("==============================================")
    print("🔊 MULTIMODAL VOICE AUTHENTICATION SYSTEM")
    print("==============================================")
    print("[R] Register new user")
    print("[A] Authenticate existing user")
    print("==============================================\n")

    mode = input("Select mode → ").lower().strip()
    if mode.startswith("r"):
        register_user()
    else:
        authenticate_user()
